# Catalyst Optimizer

**Technical Accomplishments:**
* Understanding about what is the Catalyst Optimizer?
* Understanding the different stages of the Catalyst Optimizer
* Example of Physical Plan Optimization (x2)
* Example of Predicate Pushdown

In [1]:
# Because we will need it later...
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("ColabSparkSession") \
    .getOrCreate()

## Catalyst Optimizer

* Catalyst Optimize is the fundamental to the `SQL` and `DataFrames` API.
* It is an **extensible query optimizer**.
* It actually contains a **general library for representing trees and applying rules** to manipulate them.
* Several public extension points, including external data sources and user-defined types.

<a href="https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html" target="_blank">Deep Dive into Spark SQL’s Catalyst Optimizer</a> (April 13, 2015)

Processing is broken down into several stages as we can see here:

![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

## Optimized Logical Plan

**Rewriting our code** is one of the many optimizations performed by the Catalyst Optimizer.

### Example #1: Multiple Filters

In [ ]:
from pyspark.sql.types import DoubleType
ratingDF = spark \
.read \
.parquet("spark-data/rating-huge") \
.filter(col("rating") == "3.0") \
.filter(col("movieId") == "1000") 

In [ ]:
ratingDF.explain(True)

## Projection Pruning

In [ ]:
ratingDF1 = spark \
.read \
.parquet("spark-data/rating-huge") \
.filter(col("rating") > 3.0) \
.filter(col("movieId") == "1000") \
.groupBy("rating") \
.count() \
.filter(col("rating") < 2)  \
.explain(True)

***Note:*** `explain()` is not the only way to get access to this level of detail.<br/>
But, we can also see it in the **Spark UI**.

## Columnar Predicate Pushdown

It takes place when a filter can be pushed down to the original data source, such as the database server.

For this example, we are going to compare `DataFrames` from two different sources:
* Parquet - where a predicate pushdown **WILL** take place.
* CSV - where a predicate pushdown will **NOT** take place.

In each case, we can see evidence of the pushdown (or lack of it) in the **Physical Plan**.

###  Example #1: CSV

In [ ]:
ratingCsVDF = spark.read.option("header","true").option("inferschema","true").csv("spark-data/ratings.csv") 
ratingCsVDF.printSchema()
ratingCsVDF.filter("userId =1").explain("Formatted")

In [ ]:
ratingCsVDF.write.mode("overwrite").csv("spark-data/output")

In [23]:
ratingCsVDF.filter("userId =1").write.mode("overwrite").csv("spark-data/output")

### Example2: Json

In [ ]:
ratingJsonDF = spark.read.json("spark-data/rating-json")
ratingJsonDF.printSchema()
ratingJsonDF.explain("Formatted")
ratingJsonDF.filter(col("userId") == 1).write.mode("overwrite").csv("spark-data/output")

### Example #3: Parquet

In [ ]:
ratingParquetDF = spark.read.parquet("spark-data/rating-huge") 
ratingParquetDF.printSchema()

In [ ]:
ratingParquetDF.filter(col("userId") == 1).explain("Formatted") 

In [ ]:
ratingParquetDF.filter(col("userId") == 1).write.mode("overwrite").csv("spark-data/output")

In [ ]:
ratingParquetDF1 = spark.read.parquet("spark-data/rating-typed")
ratingParquetDF1.printSchema()

In [ ]:
ratingParquetDF1.filter(col("userId") == 1).explain("Formatted") 

In [ ]:
ratingParquetDF1.filter(col("userId") == 1).write.mode("overwrite").csv("spark-data/output")

In [ ]:
ratingParquetDF1.filter(col("rating") == 3.5).write.mode("overwrite").csv("spark-data/output")

In [ ]:
ratingParquetDF1.filter(col("rating") == 3.5).explain("Formatted")

## Constant Folding

In [ ]:
ratingCSV.withColumn("newRating", lit(1+2)).explain(True)

## End of Exercise